In [2]:
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score, make_scorer
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier 

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [4]:
giventrain = pd.read_csv('train_2kmZucJ.csv')
giventest = pd.read_csv('test_oJQbWVk.csv')

In [5]:
print(giventrain.shape) #dimensions of train
print(giventest.shape)#dimensions of test

(7920, 3)
(1953, 2)


In [6]:
giventrain.columns # checking column names


Index(['id', 'label', 'tweet'], dtype='object')

In [7]:
giventest.columns # checking column names

Index(['id', 'tweet'], dtype='object')

In [8]:
# to inspect data
def inspect_data(data):
    return pd.DataFrame({"Data Type":data.dtypes,"No of Levels":data.apply(lambda x: x.unique().shape[0],axis=0), "Levels":data.apply(lambda x: str(x.unique()),axis=0)})
inspect_data(giventrain)

,Data Type,No of Levels,Levels
id,int64,7920,[1 2 3 ... 7918 7919 7920]
label,int64,2,[0 1]
tweet,object,7918,['#fingerprint #Pregnancy Test https://goo.gl/...


In [9]:
giventrain['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [10]:
giventrain.isnull().sum(axis=0)


id       0
label    0
tweet    0
dtype: int64

In [11]:
pd.DataFrame({"GTrain":giventrain.isnull().sum(),"GTest":giventest.isnull().sum()})

,GTrain,GTest
id,0,0.0
label,0,NaN
tweet,0,0.0


In [12]:
#(train == 0).sum().sum() #sum of null values 


In [13]:
## Drop columns which are not significant
noid_train=giventrain.drop(["id"],axis=1,inplace=False)

In [14]:
noid_test=giventest.drop(["id"],axis=1,inplace=False)

In [15]:
noid_train.columns

Index(['label', 'tweet'], dtype='object')

In [16]:
print(noid_train.shape) #dimensions of train
print(noid_test.shape)#dimensions of test

(7920, 2)
(1953, 1)


In [17]:
#drop id

In [18]:
## Decoupling target column

In [19]:
#y_train =noid_train['label']
#X_train = noid_train.copy().drop('label', axis=1)


In [20]:
## Train-Test Split


In [21]:
from sklearn.model_selection import train_test_split

y =noid_train['label'].tolist()
X = noid_train.loc[:,'tweet'].tolist()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=1224)

## TF-IDF Vectorization

In [22]:

tokenizer = TfidfVectorizer(ngram_range = (1,3), stop_words=None,min_df=10)
X_train_tf = tokenizer.fit_transform(X_train).toarray()
X_val_tf = tokenizer.transform(X_val).toarray()

print(X_train_tf.shape)
print(X_val_tf.shape)

(6336, 2347)
(1584, 2347)


In [23]:
X_train_tf.shape

(6336, 2347)

In [109]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2000, n_iter=10, random_state=42)
X_train_svd = svd.fit_transform(X_train_tf)
print(svd.explained_variance_ratio_.sum())
X_val_svd = svd.transform(X_val_tf)

1.0000000000000007


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report

logit = LogisticRegression()
logit.fit(X_train_tf,y_train)

## Predictions
train_logit_preds = logit.predict(X_train_tf)
val_logit_preds = logit.predict(X_val_tf)

print("Train Accuracy",logit.score(X_train_tf,y_train))
print(confusion_matrix(y_train,train_logit_preds))
print(classification_report(y_train,train_logit_preds))

print("val Accuracy",logit.score(X_val_tf,y_val))
print(confusion_matrix(y_val,val_logit_preds))
print(classification_report(y_val,val_logit_preds))

Train Accuracy 0.9188762626262627
[[4464  272]
 [ 242 1358]]
             precision    recall  f1-score   support

          0       0.95      0.94      0.95      4736
          1       0.83      0.85      0.84      1600

avg / total       0.92      0.92      0.92      6336

val Accuracy 0.8806818181818182
[[1072   86]
 [ 103  323]]
             precision    recall  f1-score   support

          0       0.91      0.93      0.92      1158
          1       0.79      0.76      0.77       426

avg / total       0.88      0.88      0.88      1584



In [120]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

NBclassifier = GaussianNB()
NBclassifier.fit(X_train_tf, y_train)

## Predictions
train_nb_preds = NBclassifier.predict(X_train_tf)
val_nb_preds = NBclassifier.predict(X_val_tf)

print("Train Accuracy",NBclassifier.score(X_train_tf,y_train))
print(confusion_matrix(y_train,train_nb_preds))
print(classification_report(y_train,train_nb_preds))

print("val Accuracy",NBclassifier.score(X_val_tf,y_val))
print(confusion_matrix(y_val,val_nb_preds))
print(classification_report(y_val,val_nb_preds))

Train Accuracy 0.7918244949494949
[[3418 1318]
 [   1 1599]]
             precision    recall  f1-score   support

          0       1.00      0.72      0.84      4736
          1       0.55      1.00      0.71      1600

avg / total       0.89      0.79      0.81      6336

val Accuracy 0.7430555555555556
[[796 362]
 [ 45 381]]
             precision    recall  f1-score   support

          0       0.95      0.69      0.80      1158
          1       0.51      0.89      0.65       426

avg / total       0.83      0.74      0.76      1584



In [ ]:
tfidf2

In [27]:

tokenizer_nb = TfidfVectorizer(ngram_range = (1,3), stop_words=None,min_df=5)
X_train_tfnb = tokenizer_nb.fit_transform(X_train).toarray()
X_val_tfnb = tokenizer_nb.transform(X_val).toarray()

print(X_train_tfnb.shape)
print(X_val_tfnb.shape)

(6336, 5302)
(1584, 5302)


In [28]:
X_test_tfnb = tokenizer_nb.transform(noid_test).toarray()


In [31]:
test_tok=tokenizer_nb.transform(noid_test["tweet"]).toarray()

In [32]:
test_tok.shape

(1953, 5302)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report

logit = LogisticRegression()
logit.fit(X_train_tfnb,y_train)

## Predictions
trainnb_logit_preds = logit.predict(X_train_tfnb)
valnb_logit_preds = logit.predict(X_val_tfnb)

print("Train Accuracy",logit.score(X_train_tfnb,y_train))
print(confusion_matrix(y_train,trainnb_logit_preds))
print(classification_report(y_train,trainnb_logit_preds))

print("val Accuracy",logit.score(X_val_tfnb,y_val))
print(confusion_matrix(y_val,valnb_logit_preds))
print(classification_report(y_val,valnb_logit_preds))

Train Accuracy 0.9264520202020202
[[4502  234]
 [ 232 1368]]
             precision    recall  f1-score   support

          0       0.95      0.95      0.95      4736
          1       0.85      0.85      0.85      1600

avg / total       0.93      0.93      0.93      6336

val Accuracy 0.8869949494949495
[[1080   78]
 [ 101  325]]
             precision    recall  f1-score   support

          0       0.91      0.93      0.92      1158
          1       0.81      0.76      0.78       426

avg / total       0.89      0.89      0.89      1584



In [34]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

NBclassifier = GaussianNB()
NBclassifier.fit(X_train_tfnb, y_train)

## Predictions
trainnb_nb_preds = NBclassifier.predict(X_train_tfnb)
valnb_nb_preds = NBclassifier.predict(X_val_tfnb)

print("Train_nb Accuracy",NBclassifier.score(X_train_tfnb,y_train))
print(confusion_matrix(y_train,trainnb_nb_preds))
print(classification_report(y_train,trainnb_nb_preds))

print("val_nb Accuracy",NBclassifier.score(X_val_tfnb,y_val))
print(confusion_matrix(y_val,valnb_nb_preds))
print(classification_report(y_val,valnb_nb_preds))

Train_nb Accuracy 0.8880997474747475
[[4027  709]
 [   0 1600]]
             precision    recall  f1-score   support

          0       1.00      0.85      0.92      4736
          1       0.69      1.00      0.82      1600

avg / total       0.92      0.89      0.89      6336

val_nb Accuracy 0.8162878787878788
[[942 216]
 [ 75 351]]
             precision    recall  f1-score   support

          0       0.93      0.81      0.87      1158
          1       0.62      0.82      0.71       426

avg / total       0.84      0.82      0.82      1584



In [35]:


#score_metric = make_scorer(f1_score,'label')
## n_jobs = -1 uses all cores of processor
## max_features is the maximum number of attributes to select for each tree
rfc_grid = RandomForestClassifier(n_jobs=-1, max_features='sqrt', class_weight='balanced',oob_score=True)
 
# Use a grid over parameters of interest
## n_estimators is the number of trees in the forest
## max_depth is how deep each tree can be
## min_sample_leaf is the minimum samples required in each leaf node for the root node to split
## "A node will only be split if in each of it's leaf nodes there should be min_sample_leaf"

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20]}
 
rfc_cv_grid = RandomizedSearchCV(estimator = rfc_grid, param_distributions = param_grid, cv = 3, n_iter=20 , n_jobs=8)
rfc_cv_grid.fit(X_train_tfnb, y_train)
rfc_cv_grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=18, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=75, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [36]:
rfc_trainnb_pred = rfc_cv_grid.best_estimator_.predict(X_train_tfnb)
rfc_valnb_pred = rfc_cv_grid.best_estimator_.predict(X_val_tfnb)


In [37]:
rfc_cv_grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=18, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=75, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [38]:

print("Train")
print(accuracy_score(y_train,rfc_trainnb_pred))
print(confusion_matrix(y_train,rfc_trainnb_pred))
print(classification_report(y_train,rfc_trainnb_pred))

print("OOB Score", rfc_cv_grid.best_estimator_.oob_score_)

print("val")
print(accuracy_score(y_val,rfc_valnb_pred))
print(confusion_matrix(y_val,rfc_valnb_pred))
print(classification_report(y_val,rfc_valnb_pred))

Train
0.8585858585858586
[[3954  782]
 [ 114 1486]]
             precision    recall  f1-score   support

          0       0.97      0.83      0.90      4736
          1       0.66      0.93      0.77      1600

avg / total       0.89      0.86      0.87      6336

OOB Score 0.849905303030303
val
0.8472222222222222
[[952 206]
 [ 36 390]]
             precision    recall  f1-score   support

          0       0.96      0.82      0.89      1158
          1       0.65      0.92      0.76       426

avg / total       0.88      0.85      0.85      1584



In [41]:
from sklearn.ensemble import RandomForestClassifier
# Model initilization
rfc_50 = RandomForestClassifier(n_estimators=50, max_depth=22, oob_score=True)
# Model training
Xrf = X_train_tfnb
yrf = y_train
rfc_50.fit(Xrf ,yrf )
# Model predictions
rfc50_train_pred = rfc_50.predict(Xrf)


In [42]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Train")
print(accuracy_score(yrf,rfc50_train_pred))
print(confusion_matrix(yrf,rfc50_train_pred))
print(classification_report(yrf,rfc50_train_pred))

print("OOB Score", rfc_50.oob_score_)



Train
0.9455492424242424
[[4669   67]
 [ 278 1322]]
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      4736
          1       0.95      0.83      0.88      1600

avg / total       0.95      0.95      0.94      6336

OOB Score 0.860479797979798


In [43]:
rfc50_val_pred = rfc_50.predict(X_val_tfnb)
print("val")
print(accuracy_score(y_val,rfc50_val_pred))
print(confusion_matrix(y_val,rfc50_val_pred))
print(classification_report(y_val,rfc50_val_pred))

val
0.8541666666666666
[[1083   75]
 [ 156  270]]
             precision    recall  f1-score   support

          0       0.87      0.94      0.90      1158
          1       0.78      0.63      0.70       426

avg / total       0.85      0.85      0.85      1584



In [ ]:
#weighted f1 score

In [44]:
# rfc_test_pred = rfc_cv_grid.best_estimator_.predict(X_test_tfnb)

In [45]:
#print("Test")
#print(accuracy_score(y_test,rfc_test_pred))
#print(confusion_matrix(y_test,rfc_test_pred))
#print(classification_report(y_test,rfc_test_pred))

# as logistic model is giving good accuracy we use it on test

In [46]:
# log_new=  pd.DataFrame({'id':test_tmp['EmployeeID'],'LeadershipPotentialScore':test_predicted_rf})

In [47]:
test_tok

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
testnb_logit_pred=logit.predict(test_tok)

In [233]:
#pd.DataFrame({
#    "ID": giventest['id'],
#    "Label": logit.predict(test_tok)
#})

In [63]:
final_tweet_pred=pd.DataFrame({
    "id": giventest['id'],
    "label": testnb_logit_pred
})

In [65]:
final_tweet_pred.head()

,id,label
0,7921,1
1,7922,0
2,7923,1
3,7924,1
4,7925,1


In [64]:
final_tweet_pred.shape

(1953, 2)

In [66]:
final_tweet_pred.to_csv("submission_tweet.csv",index=False)
